## Week 6 of Machine learning zoomcap

This week focuss on Decision Trees and Ensemble

### Load the require libary for the project

In [133]:
import pandas as pd
import numpy as np
 
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

### Download the data for the project 

- Download the data and read using wget and pandas respectively
- Per some formatting of the columns to make sure they are consistent

In [12]:
data = 'https://raw.githubusercontent.com/gastonstat/CreditScoring/refs/heads/master/CreditScoring.csv'
#!wget $data
#!head CreditScoring.csv

In [114]:
df = pd.read_csv('CreditScoring.csv')
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [115]:
df.columns = df.columns.str.lower()

In [116]:
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


#### A bit of data expolaration

In [117]:
df.describe()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
count,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4.455000e+03,4.455000e+03,4.455000e+03,4455.000000,4455.000000
mean,1.281257,7.987205,2.657015,46.441751,37.077666,1.879012,1.173513,1.675870,55.568799,7.633170e+05,1.060341e+06,4.043820e+05,1039.021773,1462.875645
std,0.450162,8.173444,1.610467,14.655225,10.984856,0.643748,0.378733,0.954035,19.515878,8.703625e+06,1.021757e+07,6.344253e+06,474.543007,628.089913
min,0.000000,0.000000,0.000000,6.000000,18.000000,0.000000,1.000000,0.000000,35.000000,0.000000e+00,0.000000e+00,0.000000e+00,100.000000,105.000000
25%,1.000000,2.000000,2.000000,36.000000,28.000000,2.000000,1.000000,1.000000,35.000000,8.000000e+01,0.000000e+00,0.000000e+00,700.000000,1117.500000
50%,1.000000,5.000000,2.000000,48.000000,36.000000,2.000000,1.000000,1.000000,51.000000,1.200000e+02,3.500000e+03,0.000000e+00,1000.000000,1400.000000
75%,2.000000,12.000000,4.000000,60.000000,45.000000,2.000000,1.000000,3.000000,72.000000,1.660000e+02,6.000000e+03,0.000000e+00,1300.000000,1692.000000
max,2.000000,48.000000,6.000000,72.000000,68.000000,5.000000,2.000000,4.000000,180.000000,1.000000e+08,1.000000e+08,1.000000e+08,5000.000000,11140.000000


In [118]:
## show all the columns as int64 eventhoug some are categorical. This need to looked into
df.dtypes

status       int64
seniority    int64
home         int64
time         int64
age          int64
marital      int64
records      int64
job          int64
expenses     int64
income       int64
assets       int64
debt         int64
amount       int64
price        int64
dtype: object

In [119]:
for col in df.columns:
    print(col)
    print(df[col].unique()[:5])
    print(df[col].nunique())
    print()

status
[1 2 0]
3

seniority
[ 9 17 10  0  1]
47

home
[1 2 5 3 6]
7

time
[60 36 12 48 18]
11

age
[30 58 46 24 26]
50

marital
[2 3 1 4 5]
6

records
[1 2]
2

job
[3 1 2 0 4]
5

expenses
[73 48 90 63 46]
94

income
[129 131 200 182 107]
353

assets
[    0  3000  2500  3500 10000]
160

debt
[   0 2500  260 2000  500]
183

amount
[ 800 1000 2000  900  310]
285

price
[ 846 1658 2985 1325  910]
1419



### Data cleaning and preparation for the decision tree 

- Clean the data to make sure it is consistent
- Take care of missing values and outliers
- Prepare categorical variables (change them from numeric to categorical)
    - Create a dictionary for each categorical value column and use it to map the values

In [120]:
df.status.value_counts()

status
1    3200
2    1254
0       1
Name: count, dtype: int64

In [121]:
## create a dictionary for the status and use it to change the values
status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

In [122]:
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,ok,9,1,60,30,2,1,3,73,129,0,0,800,846
1,ok,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,default,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,ok,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,ok,0,1,36,26,1,1,1,46,107,0,0,310,910


In [123]:
## Now do the same for the rest of the categorical varibles

In [124]:
home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}
df.home = df.home.map(home_values)
 
marital_values = {
    1: 'single', 
    2: 'married', 
    3: 'widow', 
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}
df.marital = df.marital.map(marital_values)
 
records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}
df.records = df.records.map(records_values)
 
job_values = {
    1: 'fixed', 
    2: 'partime', 
    3: 'freelance', 
    4: 'others',
    0: 'unk'
}
df.job = df.job.map(job_values)
 
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,ok,9,rent,60,30,married,no,freelance,73,129,0,0,800,846
1,ok,17,rent,60,58,widow,no,fixed,48,131,0,0,1000,1658
2,default,10,owner,36,46,married,yes,freelance,90,200,3000,0,2000,2985
3,ok,0,rent,60,24,single,no,fixed,63,182,2500,0,900,1325
4,ok,0,rent,36,26,single,no,fixed,46,107,0,0,310,910


In [125]:
## Dealing with missing values.
## First look at the data using the describe functions
## It can be seen that there are large values "99999999.0" which are extremely large (outliers). These values need to be replaced. 
df.describe().round()

,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0
mean,8.0,46.0,37.0,56.0,763317.0,1060341.0,404382.0,1039.0,1463.0
std,8.0,15.0,11.0,20.0,8703625.0,10217569.0,6344253.0,475.0,628.0
min,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,5.0,48.0,36.0,51.0,120.0,3500.0,0.0,1000.0,1400.0
75%,12.0,60.0,45.0,72.0,166.0,6000.0,0.0,1300.0,1692.0
max,48.0,72.0,68.0,180.0,99999999.0,99999999.0,99999999.0,5000.0,11140.0


In [126]:
## create a vector of the affected columns in the above table
columns_affected = ['income', 'assets', 'debt']
for col in columns_affected:
    df[col] = df[col].replace(to_replace=99999999, value=np.nan)

In [127]:
df.describe().round()

,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
mean,8.0,46.0,37.0,56.0,131.0,5403.0,343.0,1039.0,1463.0
std,8.0,15.0,11.0,20.0,86.0,11573.0,1246.0,475.0,628.0
min,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,5.0,48.0,36.0,51.0,120.0,3000.0,0.0,1000.0,1400.0
75%,12.0,60.0,45.0,72.0,165.0,6000.0,0.0,1300.0,1692.0
max,48.0,72.0,68.0,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [128]:
### look at the categorical variable again
categorical = list(df.dtypes[df.dtypes == 'object'].index)
for col in df[categorical]:
    print(col)
    print(df[col].unique())
    print(df[col].value_counts())
    print(df[col].nunique())
    print()

status
['ok' 'default' 'unk']
status
ok         3200
default    1254
unk           1
Name: count, dtype: int64
3

home
['rent' 'owner' 'parents' 'private' 'other' 'ignore' 'unk']
home
owner      2107
rent        973
parents     783
other       319
private     247
ignore       20
unk           6
Name: count, dtype: int64
7

marital
['married' 'widow' 'single' 'separated' 'divorced' 'unk']
marital
married      3241
single        978
separated     130
widow          67
divorced       38
unk             1
Name: count, dtype: int64
6

records
['no' 'yes']
records
no     3682
yes     773
Name: count, dtype: int64
2

job
['freelance' 'fixed' 'partime' 'unk' 'others']
job
fixed        2806
freelance    1024
partime       452
others        171
unk             2
Name: count, dtype: int64
5



In [129]:
## Replace the single uknown status row.
df = df[df.status != 'unk'].reset_index(drop=True)

### Split the dataset into training test and validation
- Use sklearn to split the dataset to train, test and validation

In [134]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)
 
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

#### Change target variable to 0 and 1 for the machine learning

In [141]:
y_train = (df_train.status == 'default').astype('int').values
y_val = (df_val.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_val['status']
del df_test['status']

In [143]:
df_train

,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,10,owner,36,36,married,no,freelance,75,0.0,10000.0,0.0,1000,1400
1,6,parents,48,32,single,yes,fixed,35,85.0,0.0,0.0,1100,1330
2,1,parents,48,40,married,no,fixed,75,121.0,0.0,0.0,1320,1600
3,1,parents,48,23,single,no,partime,35,72.0,0.0,0.0,1078,1079
4,5,owner,36,46,married,no,freelance,60,100.0,4000.0,0.0,1100,1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,18,private,36,45,married,no,fixed,45,220.0,20000.0,0.0,800,1600
2668,7,private,60,29,married,no,fixed,60,51.0,3500.0,500.0,1000,1290
2669,1,parents,24,19,single,no,fixed,35,28.0,0.0,0.0,400,600
2670,15,owner,48,43,married,no,freelance,60,100.0,18000.0,0.0,2500,2976


### Part 4 Decision Trees Part

In [147]:
def assess_risk(client):
    if client['records'] == 'yes':
        if client['job'] == 'parttime':
            return 'defauly'
        else:
            return 'ok'
    else:
        if client['assets'] > 6000:
            return 'ok'
        else:
            return 'default'

xi = df_train.iloc[0].to_dict()
xi
        

{'seniority': 10,
 'home': 'owner',
 'time': 36,
 'age': 36,
 'marital': 'married',
 'records': 'no',
 'job': 'freelance',
 'expenses': 75,
 'income': 0.0,
 'assets': 10000.0,
 'debt': 0.0,
 'amount': 1000,
 'price': 1400}

In [148]:
assess_risk(xi)

'ok'

In [146]:
## import scikit-lean modules for the learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

In [153]:
### Make the data to dictionary and train the model
train_dict = df_train.fillna(0).to_dict(orient='records')
train_dict[:5]

[{'seniority': 10,
  'home': 'owner',
  'time': 36,
  'age': 36,
  'marital': 'married',
  'records': 'no',
  'job': 'freelance',
  'expenses': 75,
  'income': 0.0,
  'assets': 10000.0,
  'debt': 0.0,
  'amount': 1000,
  'price': 1400},
 {'seniority': 6,
  'home': 'parents',
  'time': 48,
  'age': 32,
  'marital': 'single',
  'records': 'yes',
  'job': 'fixed',
  'expenses': 35,
  'income': 85.0,
  'assets': 0.0,
  'debt': 0.0,
  'amount': 1100,
  'price': 1330},
 {'seniority': 1,
  'home': 'parents',
  'time': 48,
  'age': 40,
  'marital': 'married',
  'records': 'no',
  'job': 'fixed',
  'expenses': 75,
  'income': 121.0,
  'assets': 0.0,
  'debt': 0.0,
  'amount': 1320,
  'price': 1600},
 {'seniority': 1,
  'home': 'parents',
  'time': 48,
  'age': 23,
  'marital': 'single',
  'records': 'no',
  'job': 'partime',
  'expenses': 35,
  'income': 72.0,
  'assets': 0.0,
  'debt': 0.0,
  'amount': 1078,
  'price': 1079},
 {'seniority': 5,
  'home': 'owner',
  'time': 36,
  'age': 46,
  'm

In [155]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dict)

In [156]:
dv.get_feature_names_out()

array(['age', 'amount', 'assets', 'debt', 'expenses', 'home=ignore',
       'home=other', 'home=owner', 'home=parents', 'home=private',
       'home=rent', 'home=unk', 'income', 'job=fixed', 'job=freelance',
       'job=others', 'job=partime', 'job=unk', 'marital=divorced',
       'marital=married', 'marital=separated', 'marital=single',
       'marital=unk', 'marital=widow', 'price', 'records=no',
       'records=yes', 'seniority', 'time'], dtype=object)

### Part 5 Decision Trees and Ensemble Learning

In [157]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [161]:
val_dict = df_val.fillna(0).to_dict(orient='records')
X_val = dv.transform(val_dict)
y_pred = dt.predict_proba(X_val)[:,1]
roc_auc_score(y_val, y_pred)

0.6605434598338621

#### Overfitting
- Depth of growth of the tree is important
- Should not be allow to grow too deep
    - leads to overfitting

In [170]:
y_pred = dt.predict_proba(X_train)[:,1]
roc_auc_score(y_train, y_pred)

0.7761016984958594

In [171]:
dt = DecisionTreeClassifier(max_depth=3)
dt.fit(X_train, y_train)

y_pred = dt.predict_proba(X_train)[: ,1]
auc = roc_auc_score(y_train, y_pred)
print('train: ', auc)

y_pred = dt.predict_proba(X_val)[: ,1]
auc = roc_auc_score(y_val, y_pred)
print('val: ', auc)

train:  0.7761016984958594
val:  0.7389079944782155


#### Decision stump
- Decision tree of level one is called Decision Stump.
    - Not acually a tree because it is only one condition

In [189]:
## Change the max_depth and re-run the cells below to the see the features & their values
dt = DecisionTreeClassifier(max_depth=4)
dt.fit(X_train, y_train)

y_pred = dt.predict_proba(X_train)[: ,1]
auc = roc_auc_score(y_train, y_pred)
print('train: ', auc)

y_pred = dt.predict_proba(X_val)[: ,1]
auc = roc_auc_score(y_val, y_pred)
print('val: ', auc)

train:  0.8164293560926146
val:  0.7612827250490422


#### Visualizing Decision Stump

In [190]:
from sklearn.tree import export_text
print(export_text(dt))

|--- feature_25 <= 0.50
|   |--- feature_27 <= 6.50
|   |   |--- feature_1 <= 862.50
|   |   |   |--- feature_27 <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- feature_27 >  0.50
|   |   |   |   |--- class: 0
|   |   |--- feature_1 >  862.50
|   |   |   |--- feature_2 <= 8250.00
|   |   |   |   |--- class: 1
|   |   |   |--- feature_2 >  8250.00
|   |   |   |   |--- class: 0
|   |--- feature_27 >  6.50
|   |   |--- feature_12 <= 103.50
|   |   |   |--- feature_2 <= 4500.00
|   |   |   |   |--- class: 1
|   |   |   |--- feature_2 >  4500.00
|   |   |   |   |--- class: 0
|   |   |--- feature_12 >  103.50
|   |   |   |--- feature_4 <= 89.00
|   |   |   |   |--- class: 0
|   |   |   |--- feature_4 >  89.00
|   |   |   |   |--- class: 1
|--- feature_25 >  0.50
|   |--- feature_16 <= 0.50
|   |   |--- feature_12 <= 74.50
|   |   |   |--- feature_2 <= 4250.00
|   |   |   |   |--- class: 0
|   |   |   |--- feature_2 >  4250.00
|   |   |   |   |--- class: 0
|   |   |--- feature_12 >  74

In [191]:
names = dv.get_feature_names_out().tolist()
print(export_text(dt, feature_names=names))

|--- records=no <= 0.50
|   |--- seniority <= 6.50
|   |   |--- amount <= 862.50
|   |   |   |--- seniority <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- seniority >  0.50
|   |   |   |   |--- class: 0
|   |   |--- amount >  862.50
|   |   |   |--- assets <= 8250.00
|   |   |   |   |--- class: 1
|   |   |   |--- assets >  8250.00
|   |   |   |   |--- class: 0
|   |--- seniority >  6.50
|   |   |--- income <= 103.50
|   |   |   |--- assets <= 4500.00
|   |   |   |   |--- class: 1
|   |   |   |--- assets >  4500.00
|   |   |   |   |--- class: 0
|   |   |--- income >  103.50
|   |   |   |--- expenses <= 89.00
|   |   |   |   |--- class: 0
|   |   |   |--- expenses >  89.00
|   |   |   |   |--- class: 1
|--- records=no >  0.50
|   |--- job=partime <= 0.50
|   |   |--- income <= 74.50
|   |   |   |--- assets <= 4250.00
|   |   |   |   |--- class: 0
|   |   |   |--- assets >  4250.00
|   |   |   |   |--- class: 0
|   |   |--- income >  74.50
|   |   |   |--- seniority <= 5.50
|   | 

### Part 6 Decision Tree Learning Algorithm 

In [192]:
data = [
    [8000, 'default'],
    [2000, 'default'],
    [   0, 'default'],
    [5000, 'ok'],
    [5000, 'ok'],
    [4000, 'ok'],
    [9000, 'ok'],
    [3000, 'default'],
]
df_example = pd.DataFrame(data, columns=['assets', 'status'])
df_example

,assets,status
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default


In [193]:
df_example.sort_values('assets')

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


In [196]:
Ts = [0, 2000, 3000, 4000, 5000, 8000]

In [198]:
from IPython.display import display

In [203]:
for T in Ts:
    print(T)
    df_left = df_example[df_example.assets <= T]
    df_right = df_example[df_example.assets > T]

    display(df_left.sort_values('assets'))
    display(df_right.sort_values('assets'))

    print()

0


,assets,status
2,0,default


,assets,status
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok



2000


,assets,status
2,0,default
1,2000,default


...

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok


,assets,status
0,8000,default
6,9000,ok



8000


,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default


,assets,status
6,9000,ok


In [208]:
T = 4000
df_left = df_example[df_example.assets <= T]
df_right = df_example[df_example.assets > T]
 
display(df_left.sort_values('assets'))
display(df_right.sort_values('assets'))

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok


,assets,status
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


#### Misclassification Rate

In [209]:
T = 4000
df_left = df_example[df_example.assets <= T]
df_right = df_example[df_example.assets > T]
 
display(df_left.sort_values('assets'))
print(df_left.status.value_counts(normalize=True))
display(df_right.sort_values('assets'))
print(df_right.status.value_counts(normalize=True))

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok


status
default    0.75
ok         0.25
Name: proportion, dtype: float64


,assets,status
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


status
ok         0.75
default    0.25
Name: proportion, dtype: float64


In [210]:
for T in Ts:
    print(T)
    df_left = df_example[df_example.assets <= T]
    df_right = df_example[df_example.assets > T]

    display(df_left.sort_values('assets'))
    print(df_left.status.value_counts(normalize=True))
    display(df_right.sort_values('assets'))
    print(df_right.status.value_counts(normalize=True))

    print()

0


,assets,status
2,0,default


status
default    1.0
Name: proportion, dtype: float64


,assets,status
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


status
ok         0.571429
default    0.428571
Name: proportion, dtype: float64

2000


...

status
default    0.5
ok         0.5
Name: proportion, dtype: float64

8000


,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default


status
default    0.571429
ok         0.428571
Name: proportion, dtype: float64


,assets,status
6,9000,ok


status
ok    1.0
Name: proportion, dtype: float64



#### Impurity
- The misclassification rate is just one way of measuring impurity.

### Decision Tree Learning Algorithm - Part 2/2